<a href="https://colab.research.google.com/github/unknown-jun/First_step_of_data_analist/blob/master/Titanic_for_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리를 불러옴

import pandas as pd
import seaborn as sns

# 1. 데이터 준비

In [2]:
# 데이터 준비 - seaborn에서 제공하는 titanic 데이터셋 가져오기
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


**Titanic dataset columns에 대한 설명**

- survived: 생존했으면 1로 표기, 사망했으면 0으로 표기
- pclass: 유람선 객실 등급
- sex: 성별
- age: 나이
- sibsp: 동승 형제나 자매, 배우자의 수
- parch: 함께 탑승한 부모 또는 자녀의 수
- fare: 티켓 요금
- embarked : 탑승장소 S=Southhampton, C=Cherbourg, Q=Queenstown
- carbin: 선실번호


# 2. 데이터 탐색 / 전처리

In [3]:
df.info() # 데이터의 자료형, 인덱스를 확인한다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [4]:
df=df.drop(['deck', 'embark_town'], axis=1) # NaN 값이 많은 deck/ embarked와 자료가 겹치는 embark_town을 삭제
df.columns.values  # 제대로 삭제되었는지 확인

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [5]:
df['age'] = df['age'].fillna(round(df['age'].mean())) # age 컬럼의 나이를 평균값으로 대체

대체하고 나서 깨달은 사실
- 만약 대체하기 전에 boxplot을 그려봐서 평균값으로 대체하는게 맞는 것인지 확인했다면 더 좋았을 것 같다

In [6]:
most_freq = df['embarked'].value_counts(dropna=True).idxmax() # embarked 컬럼에 최빈값을 구한다
most_freq                                                     # dropna를 True로 하여 NaN 값 무시

'S'

In [7]:
df['embarked'].fillna(most_freq, inplace=True)  # embarked의 NaN값을 위에서 구한 최빈값으로 대체한다

# 3. 분석에 사용할 속성 선택

In [8]:
df = df[['survived', 'pclass','sex','age','sibsp','parch','embarked']] # 분석에 활용할 columns만 선택한다.
df

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,S
887,1,1,female,19.0,0,0,S
888,0,3,female,30.0,1,2,S
889,1,1,male,26.0,0,0,C


In [9]:
onehot_sex = pd.get_dummies(df['sex'])     # sex column을 더미변수로 변경하여 변수 생성
df = pd.concat([df,onehot_sex], axis=1)    # df Dataframe에 합침

onehot_embarked = pd.get_dummies(df['embarked'], prefix='town') 
# embarked column을 더미변수로 지정하되 prefix를 town으로 지정하여 town별로 더미변수를 만들도록 지정
df = pd.concat([df, onehot_embarked], axis=1)  # df Dataframe에 합침

df.drop(['sex','embarked'], axis=1, inplace=True) # one hot encoding으로 중복된 컬럼을 지움
df.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


# 4. 훈련/검증 데이터 분할
- 데이터셋 구분 - 훈련용(train data) / 검증용(test data)

In [12]:
x = df[['pclass', 'age', 'sibsp', 'parch','female','male',
        'town_C', 'town_Q', 'town_S']]  # 독립변수
y = df[['survived']]   # 종속변수

# 설명변수 데이터를 정규화(normalization)
from sklearn import preprocessing   
# 정규화를 위해 sklearn의 preprocseeing 를 import 한다
x = preprocessing.StandardScaler().fit(x).transform(x)
# 스케일링 중 평균과 표준편차로 스케일링 한다.
# StandardScaler는 이상치에 영향을 많이 받는다.
# fit() : 데이터 변환을 학습, 학습용 데이터에만 적용
# transform() : 실제 데이터의 스케일을 조정, 학습용 데이터와 테스트 데이터에 적용

In [13]:
from sklearn.model_selection import train_test_split
# 데이터를 훈련/검증 데이터로 분할하기 위한 모듈 import
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=10)
# train data와 test data로 구분(7:3 비율)

In [14]:
print('train data 개수: ', x_train.shape)
print('test data 개수: ', x_test.shape)

train data 개수:  (623, 9)
test data 개수:  (268, 9)


# 5. 모형 학습 및 검증
- knn 분류 모형을 사용하여 모형을 학습하고 검증한다.

In [18]:
from sklearn.neighbors import KNeighborsClassifier
# sklearn 라이브러리에서 KNN 분류모형 가져오기

knn = KNeighborsClassifier(n_neighbors=5)
# 모형 객체 생성(k=5로 설정)

knn.fit(x_train, y_train)
# train data를 가지고 모형 학습

y_hat = knn.predict(x_test)
# test data를 가지고 y_HAT을 예측(분류)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 0 1 1 0 0 0 0 0]
[[0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


In [19]:
from sklearn import metrics
# 모형 성능 평가를 위한 Confusion Matrix를 이용하기 위해 sklearn의 metrics를 import
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
knn_matrix

array([[152,  22],
       [ 27,  67]])

In [21]:
# 모형 성능평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       174
           1       0.75      0.71      0.73        94

    accuracy                           0.82       268
   macro avg       0.80      0.79      0.80       268
weighted avg       0.82      0.82      0.82       268



# 평가
- 사망자 예측도는 0.86이고
- 생존자 예측도는 0.73으로 측정되었다.
<br>
- 만약 나이를 평균값으로 치환하지 않았다면
- 사망자 예측도는 0.84이며
- 생존자 예측도는 0.76이었다.
